In [1]:
import streamlit as st
import pandas as pd
import os

# -------------------------
# Onglets Streamlit
# -------------------------
tabs = st.tabs(["➕ Saisie des résultats", "📊 Statistiques"])

# Onglet 1 : Saisie des résultats
with tabs[0]:
    st.header("Saisie d'un nouveau résultat")

    joueurs = list(set(df["joueur_A"]).union(set(df["joueur_B"]))) if not df.empty else []
    new_joueur_A = st.text_input("Joueur A")
    new_joueur_B = st.text_input("Joueur B")
    score_A = st.number_input("Score A", min_value=0, max_value=13, value=0)
    score_B = st.number_input("Score B", min_value=0, max_value=13, value=0)

    if st.button("Enregistrer le résultat"):
        if new_joueur_A and new_joueur_B and new_joueur_A != new_joueur_B:
            vainqueur = new_joueur_A if score_A > score_B else new_joueur_B
            new_data = pd.DataFrame([[new_joueur_A, new_joueur_B, score_A, score_B, vainqueur]],
                                    columns=df.columns)
            df = pd.concat([df, new_data], ignore_index=True)
            df.to_csv(DATA_FILE, index=False)
            st.success("Résultat enregistré !")
        else:
            st.error("Vérifiez les noms des joueurs")

# Onglet 2 : Statistiques
with tabs[1]:
    st.header("Classement et statistiques")

    if df.empty:
        st.info("Aucun résultat enregistré pour l'instant.")
    else:
        # Calcul des stats par joueur
        joueurs = set(df["joueur_A"]).union(set(df["joueur_B"]))
        stats = []
        for j in joueurs:
            jouees = len(df[(df["joueur_A"] == j) | (df["joueur_B"] == j)])
            gagnees = len(df[df["vainqueur"] == j])
            perdues = jouees - gagnees
            points_marques = df.loc[df["joueur_A"] == j, "score_A"].sum() + df.loc[df["joueur_B"] == j, "score_B"].sum()
            points_encaisses = df.loc[df["joueur_A"] == j, "score_B"].sum() + df.loc[df["joueur_B"] == j, "score_A"].sum()
            goal_average = points_marques - points_encaisses
            stats.append([j, jouees, gagnees, perdues, round(gagnees/jouees*100,1) if jouees>0 else 0, points_marques, points_encaisses, goal_average])

        stats_df = pd.DataFrame(stats, columns=["Joueur", "Parties jouées", "Victoires", "Défaites", "% Victoires", "Points marqués", "Points encaissés", "Goal Average"])
        stats_df = stats_df.sort_values(by=["Victoires", "Goal Average"], ascending=[False, False])

        st.dataframe(stats_df, use_container_width=True)

        # Graphique simple
        st.bar_chart(stats_df.set_index("Joueur")["Victoires"])